# E91 protocol

This notebook illustrates a simplified implementation of the [E91](https://en.wikipedia.org/wiki/Quantum_key_distribution#E91_protocol:_Artur_Ekert_.281991.29) protocol using the [qiskit](https://qiskit.org/) framework.
The steps involved can be summarized as follows:

1. creation of entangled states (encoding)
2. generation of raw key (decoding)
3. sifting
4. CHSH inequality test (parameter estimation)

For each step, a class performing the required operations has been implemented.

Let's start with the necessary imports:

In [1]:
import sys  
sys.path.insert(0, '../src')

In [2]:
from e91.encoding import Encoder, EncodingCircuit
from e91.decoding import Decoder, DecodingCircuit
from e91.sifting import Sifter
from e91.estimation import Estimator
from random import choices

In [3]:
length = 5000 # use a smaller value to speed up the computation

## 1. Creation of entangled states
Alice and Bob will receive each `length` qubit of the pair prepared in the state $\frac{|01> - |10>}{\sqrt{2}}$.   
The states are generated using the following circuit:

In [4]:
EncodingCircuit().circuit.draw('text')

┌───┐┌───┐      ░ 
q_0: ┤ X ├┤ H ├──■───░─
     ├───┤└───┘┌─┴─┐ ░ 
q_1: ┤ X ├─────┤ X ├─░─
     └───┘     └───┘ ░

The class to which is delegated the creation of `length` of these states is `e91.encoding.Encoder`:

In [5]:
encoder = Encoder()
states = encoder.encode(length)

Since to store `lenght` actual state vectors, $2^\text{lenght}$ float are required, `states` contains the circuits that produce that states

## 2. Generation of raw key
Let's imagine that the qubits had been distributed to Alice and Bob.  
Both of them choose `length` random bases among the bases of operators $A_0$, $A_1$, $A_2$ (Alice), $B_0$, $B_1$, $B_2$ (Bob)(these operators are specified by the protocol) in which measure their qubits:

In [6]:
a_bases = choices((0,1,2), k=length)
b_bases = choices((0,1,2), k=length)
#print(f'a_bases : {a_bases}') # [0,0,1,2,1,...]
#print(f'b_bases : {b_bases}') # [1,2,1,0,1,...]

Alice and Bob then measure their qubits using the chosen bases.   

Since in qiskit is possible to measure only in the computational basis, a measure operation is preceded by an appropriate unitary transformation.  

For instance, if `a_bases[i]` = 1 and `b_bases[i]` = 2, the circuit used to perform the measure is:

In [7]:
DecodingCircuit(1,2).circuit.draw('text')

┌───┐     ░ ┌─┐   
q_0: ───┤ H ├─────░─┤M├───
     ┌──┴───┴───┐ ░ └╥┘┌─┐
q_1: ┤ Ry(-π/4) ├─░──╫─┤M├
     └──────────┘ ░  ║ └╥┘
a: 1/════════════════╩══╬═
                     0  ║ 
                        ║ 
b: 1/═══════════════════╩═
                        0

The class to which is delegated the operations involved in measuring each of the `length` qubits is `e91.decoding.Decoder`  
The result of `decode` operation is a string of bits (the raw key), for each partecipant:

In [8]:
decoder = Decoder()
a_raw_key, b_raw_key = decoder.decode(a_bases, b_bases, states)
#print(f'a_raw_key : {a_raw_key.bin}') # 10101...
#print(f'b_raw_key : {b_raw_key.bin}') # 01101...

## 3. Sifting
Let's imagine that Alice transmitted the list of her chosen bases to Bob, and vice versa.
They can now generate the sifted keys, starting from their raw keys, and using the bases where `a_bases[i] == b_bases[i]` and `a_bases[i] == 0` or `a_bases[i] == 2`   

The sifting is done using the class `e91.sifting.Sifter`

Since this simplified implementation of the protocol is ideal, Alice's and Bob's sifted key will be one the 1 complement of the other; furthermore, the ratio of raw and sifted keys's  lengths will be close to $\frac{2}{9}$

In [9]:
sifter = Sifter(a_bases, b_bases)
a_sifted_key = sifter.sift(a_raw_key)
b_sifted_key = sifter.sift(b_raw_key)
#print(f'a_sifted_key : {a_sifted_key.bin}')
#print(f'b_sifted_key : {b_sifted_key.bin}')
print(f'Are the keys complementary? {a_sifted_key == ~b_sifted_key}')
print(f'sifted_key/raw_key lengths ratio: {len(a_sifted_key)/len(a_raw_key)} (expected: {2/9})')

Are the keys complementary? True
sifted_key/raw_key lengths ratio: 0.2292 (expected: 0.2222222222222222)


## 3. CHSH inequality test
Alice and Bob now have to make sure thate an eavesdropper, listening to their public communication, has not acquired information about the keys.   
They do so verifying that the quantity $<A_0B_2> + <A_0B_1> + <A_1B_2> - <A_1B_1>$ is (nearly) equal to $-2\sqrt{2}$.   
The expectation values $<A_iB_j>$ are computed simply counting each occurence of the possible measure pairs (0,0), (0,1), (1,0), (1,1) (using the raw keys) for each operator pair $A_iB_j$ (determined by the Alice's and Bob's chosen bases).   

These operaions are implemented in the class `e91.estimation.Estimator`

In [10]:
estimator = Estimator()
parameter = estimator.estimate(a_raw_key, b_raw_key, a_bases, b_bases)
print(f'parameter: {parameter} (expected : {-2 * pow(2,0.5)})')

parameter: -2.8362541624906896 (expected : -2.8284271247461903)
